In [ ]:
#using intel D435 realsense to test image_segmentation
#model: segment_trt_fp16.pth
#output: [depthframe_image, colorframe_image, seg_image]
#or output: [colorframe_image, seg_image]
#last updated: 11/10/2021

In [ ]:
#load the model
from torch2trt import TRTModule
import torch
model_trt = TRTModule()
model_trt.load_state_dict(torch.load('segment_trt_fp16.pth'))

In [ ]:
#import module for display
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg
import ipywidgets

In [ ]:
#import key function.py
from module_D435_segmentation.function_D435_segmentation import *

In [ ]:
#display and create the image boxes
seg_image = ipywidgets.Image(format='jpeg', width=224, height=224)
#depthframe_image = ipywidgets.Image(format='jpeg', width=224, height=224)
colorframe_image = ipywidgets.Image(format='jpeg', width=224, height=224)
#display(widgets.VBox([widgets.HBox([depthframe_image, colorframe_image, seg_image]),]))
display(widgets.VBox([widgets.HBox([ colorframe_image, seg_image]),]))

In [ ]:
#load images from D435
dc = DepthCamera()
#ret, depth_frame, color_frame = dc.get_frame()
ret, color_frame = dc.get_frame()

In [ ]:
#execution for segmentation
def execute(change):
    image = change['new']
    #ret, depth_frame, color_frame = dc.get_frame()
    ret, color_frame = dc.get_frame()
    output = model_trt(preprocess(color_frame).half())[0].detach().cpu().float().numpy()
    mask=decode_segmap(output.argmax(0))
    seg_image.value = bgr8_to_jpeg(mask)
    
    #image of D435 generation
    #depthframe_image.value = bgr8_to_jpeg(depth_frame)
    colorframe_image.value = bgr8_to_jpeg(color_frame)

In [ ]:
#run real time of the images
while True:
    mask = execute({'new': color_frame})


In [ ]:
# #numpy.ndarray
print(color_frame)
# print(color_frame.size)
# print(color_frame.shape)
# print(color_frame.dtype)
# print(color_frame.astype)
# print(color_frame.ctypes)
# print(color_frame.ndim)

In [ ]:

# from jetbot import Camera
# color_frame.observe(execute, names='value')